In [1]:
import os
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import numpy as np

In [14]:

#Calcular beta de una acción respecto al S&P 500
def calcular_beta(ticker):

    sp500 = yf.download('^GSPC', start='2020-06-01',interval='1mo',auto_adjust=True)

    # Descargar datos de la acción
    accion = yf.download(ticker, start='2020-06-01',interval='1mo',auto_adjust=True)
    
    # Calcular retornos diarios
    sp500['Retorno_SPY'] = sp500['Close'].pct_change()
    accion['Retorno_MF'] = accion['Close'].pct_change()
    
    # Eliminar filas con valores NaN
    data = pd.concat([sp500['Retorno_SPY'], accion['Retorno_MF']], axis=1).dropna()
    display(data.head())
    # Ajustar el modelo de regresión lineal
    modelo = sm.OLS(data['Retorno_MF'], sm.add_constant(data['Retorno_SPY'])).fit()
    display(modelo.summary())
    return modelo.params[1],modelo  # Retorna la beta

In [16]:
beta,modelo=calcular_beta('WMT')
print(beta)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Retorno_SPY,Retorno_MF
Date,,
2020-07-01,0.055101,0.080314
2020-08-01,0.070065,0.073029
2020-09-01,-0.039228,0.011777
2020-10-01,-0.027666,-0.008291
2020-11-01,0.107546,0.101189


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Retorno_MF   R-squared:                       0.283
Model:                            OLS   Adj. R-squared:                  0.272
Method:                 Least Squares   F-statistic:                     24.13
Date:                Wed, 03 Sep 2025   Prob (F-statistic):           7.08e-06
Time:                        18:43:42   Log-Likelihood:                 102.23
No. Observations:                  63   AIC:                            -200.5
Df Residuals:                      61   BIC:                            -196.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0090      0.006      1.412      0.163      -0.004       0.022
Retorno_SPY     0.6603      0.134      4.912      0.000       0.392       0.929
==============================================================================
Omnibus:                       11.281   Durbin-Watson:                   2.164
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               14.673
Skew:                          -0.692   Prob(JB):                     0.000651
Kurtosis:                       4.917   Cond. No.                         22.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

0.660298542414401


/var/folders/jr/t_k402pj3bg57t7p56rgmknr0000gn/T/ipykernel_43015/3654500752.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return modelo.params[1],modelo  # Retorna la beta


b) 
- H0: que $ \beta=1 $
- H1: que $ \beta <>1$

Si el $\beta=1$ entonces la regresion lineal queda $ \hat Y = 1 * X $, lo que implica que Y sigue en promedio el índice GSPC, con lo cual, si el índice sube 1% WMT sube 1%. 

El test que queremos hacer tiene el siguiente estadístico: 
$$
    t=\frac{(\hat \beta-\beta_0)}{ee(\hat\beta)}
$$

Lo que sigue una distribución t con n-2 grados de libertad siendo n=62, por lo tanto sigue t=61

In [31]:
#B testeo la hipotes nula de que el beta obtenido en el punto anterior de esta regresion es igual a 1, con un nivel de significancia del 5%.
from scipy import stats

def testear_hipotesis(beta,modelo):
    # Estadístico de prueba
    h0=1
    df=modelo.df_resid
    estadistico = (beta - h0) / modelo.bse['Retorno_SPY'] 
    p_value = 2 * stats.t.cdf(estadistico, df)  if estadistico < 0 else 2 * (1 - stats.t.cdf(estadistico, df))
    print(p_value)
    return p_value

p_value = testear_hipotesis(beta,modelo)
print(f"p-value: {p_value}")

if p_value < 0.05:
    print("Rechazamos H0: beta es diferente de 1")
else:
    print("No rechazamos H0: beta es igual a 1")




0.014105366506379852
p-value: 0.014105366506379852
Rechazamos H0: beta es diferente de 1
